# Grafon

### Qué va a analizar nuestro grafo?

Analizar "gente que haya tenido experiencias facultativas similares" -> sacamos el factor temporal

### Qué queremos responder?

Qué electivas curso?

### Cómo es el grafo?

- Nodos: usuarios
- Aristas: cursar misma materia y "les fue parecido" -> -1 (en el final), 4-5, 6-7, 8-10
- Peso: porcentaje de similitud de materia+nota

### Dos formas de armar el análisis
- sólo correr sobre materias electivas
- correr sobre todas las materias, pero filtrar el output por electivas

### Ejemplo de conexión:

X y yo cursamos ...